In [1]:
!pip install transformers diffusers accelerate torch torchvision
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install --upgrade diffusers==0.14.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.2
    Uninstalling diffusers-0.32.2:
      Successfully uninstalled diffusers-0.32.2


In [3]:
!pip install --upgrade huggingface_hub


In [4]:
!pip uninstall -y huggingface_hub diffusers
!pip install huggingface_hub diffusers


Found existing installation: huggingface-hub 0.29.3
Uninstalling huggingface-hub-0.29.3:
  Successfully uninstalled huggingface-hub-0.29.3
Found existing installation: diffusers 0.14.0
Uninstalling diffusers-0.14.0:
  Successfully uninstalled diffusers-0.14.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.4 MB/s eta 0:00:00


In [5]:
from diffusers import StableDiffusionXLPipeline


In [ ]:
from huggingface_hub import login

login("your access token here")

In [7]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import numpy as np
import random
from google.colab import drive

drive.mount('/content/drive')

model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

home_decor_items = [
    "sofa", "lights", "vases", "curtains", "paintings", "coffee table",
    "bed", "bookshelves", "rugs", "armchairs", "side tables", "floor lamps",
    "chandeliers", "wall art", "pillow", "outdoor furniture", "shelves"
]

aesthetic_prompts = [
    "a cozy living room with a leather sofa, soft lighting, plants, and a coffee table",
    "a modern bedroom with a large bed, bedside lamps, a rug, and indoor plants",
    "a minimalist living room with a sectional sofa, pendant lights, and abstract wall art",
    "a chic dining room with a wooden dining table, dining chairs, and a large pendant chandelier",
    "a rustic kitchen with open shelving, plants, and wooden accents",
    "a luxury bathroom with marble countertops, candles, towels, and a decorative mirror",
    "a stylish home office with a desk, ergonomic chair, bookshelves, and floor lamps",
    "a cozy reading nook with a comfortable armchair, side table, and soft lighting",
    "a modern entryway with a console table, plants, and decorative mirrors",
    "a vibrant outdoor patio with outdoor furniture, string lights, and plants"
]

def generate_image(prompt):
    image = pipe(prompt).images[0]
    return image

def create_collage(images, grid_size=(2, 3), image_size=(512, 512)):
    collage_width = grid_size[1] * image_size[0]
    collage_height = grid_size[0] * image_size[1]

    collage = Image.new("RGB", (collage_width, collage_height))

    x_offset = 0
    y_offset = 0

    for i, img in enumerate(images):
        img = img.resize(image_size)
        collage.paste(img, (x_offset, y_offset))

        x_offset += image_size[0]
        if x_offset >= collage_width:
            x_offset = 0
            y_offset += image_size[1]

    return collage

def generate_collage(num_images=6):
    items = random.sample(home_decor_items, num_images)
    random_prompt = random.choice(aesthetic_prompts)
    prompt = f"{random_prompt} with home decor items like {', '.join(items)}"

    generated_images = [generate_image(prompt) for _ in range(num_images)]

    collage = create_collage(generated_images, grid_size=(2, 3), image_size=(512, 512))

    return collage

for i in range(10):
    collage = generate_collage(num_images=6)
    collage.show()
    collage.save(f"/content/drive/MyDrive/lifesetting_collage_{i+1}.jpg")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
pipe.save_pretrained("productdecor")